# Run OpenQuake scenarios sensitivity analysis

## Configuration and user input

In [ ]:
# Import dependencies
import os, glob
import oq_run_many as oq
from oq_parse_gmfs import oq_get_gmfs, write_gmf_csv


# Event name
event = '20030521_M6.8_Boumerdes'

# Indicate ruptures to be considered in calculations
ruptures_to_include = ['USGS', 'BelabbesEtAl2009', 'SemmaneEtAl2005']

# Ground Motion Models or GMPE's to include.
gmm_sources_to_include = ['GEM', 'USGS']

## Folders and files

In [ ]:
# Find all rupture files
all_rups = glob.glob(os.path.join('..', '*', event,
                                  'Rupture',
                                  'earthquake_rupture_model_*'))

# Verify that ruptures exist in folder
ruptures = []
for author in ruptures_to_include:
    exist = False 
    for rup in all_rups:
        if author in rup:
            exist = True
            ruptures.append(rup)
    if not exist:
        raise ValueError(f'No rupture found for author {author}!')

# OQ rupture paths
oq_rups = [f'../Rupture/{os.path.basename(rup)}' for rup in ruptures]

# OQ gmlt paths
oq_gmlt = [f'gmpe_logic_tree_{gmlt}.xml' for gmlt in gmm_sources_to_include]

# Event folder
country = ruptures[0].split(os.sep)[1]
root_folder = ruptures[0][:ruptures[0].find('Rupture')]
oq_folder = os.path.join(root_folder, 'OpenQuake_gmfs')
oq_sens = os.path.join(oq_folder, 'Sensitivity')

# Create Sensitivity folder
if os.path.isdir(oq_sens):
    print('Sensitivity folder already exists\n')
else:
    os.mkdir(oq_sens)
    print('Sensitivity folder created\n')


print('GMLT paths for OQ:', oq_gmlt)
print('Rupture paths for OQ:')
oq_rups


## Run calculation without conditioning the GMFs

### Stations: None

In [ ]:
# Job.ini to be use in the sensitivity analysis
job_ini = os.path.join(oq_folder, 'job_stations_none.ini')

# Run OpenQuake calculations
jobs = oq.main(job_ini, oq_rups, oq_gmlt, concurrent_jobs=1)

# For each calculation, save gmfs in csv (only for PGA)
for job in jobs:
    calc_id = job.calc_id
    df, txt = oq_get_gmfs(calc_id)
    out_df = write_gmf_csv(df, 'PGA', comment=txt, 
                           out_path=oq_sens, suffix=calc_id)

### Stations: Seismic

In [ ]:
# Job.ini to be use in the sensitivity analysis
job_ini = os.path.join(oq_folder, 'job_stations_seismic.ini')

# Run OpenQuake calculations
jobs = oq.main(job_ini, oq_rups, oq_gmlt, concurrent_jobs=1)

# For each calculation, save gmfs in csv (only for PGA)
for job in jobs:
    calc_id = job.calc_id
    df, txt = oq_get_gmfs(calc_id)
    out_df = write_gmf_csv(df, 'PGA', comment=txt, 
                           out_path=oq_sens, suffix=calc_id)

### Stations: All (i.e., seismic and macroseismic stations)

In [ ]:
# # Job.ini to be use in the sensitivity analysis
# job_ini = os.path.join(oq_folder, 'job_stations_all.ini')

# # Run OpenQuake calculations
# jobs = oq.main(job_ini, oq_rups, oq_gmlt, concurrent_jobs=1)

# # For each calculation, save gmfs in csv (only for PGA)
# for job in jobs:
#     calc_id = job.calc_id
#     df, txt = oq_get_gmfs(calc_id)
#     out_df = write_gmf_csv(df, 'PGA', comment=txt, 
#                            out_path=oq_sens, suffix=calc_id)